## Multi-class classification of Cofacts articles

This notebook is develpoed for classifying article tags defined by Cofacts.


### Tags definition

![Defintion](https://github.com/cofacts/rumors-ai/blob/master/data_exploration/img/Tags_definition.png?raw=true)

### Import data and preprocessing

In [0]:
!cp /content/drive/My\ Drive/Datasets/cofacts_20200324.zip ./

In [2]:
!unzip -o cofacts_20200324.zip

Streaming output truncated to the last 5000 lines.
  inflating: raw_data/28156.json     
  inflating: __MACOSX/raw_data/._28156.json  
  inflating: raw_data/25764.json     
  inflating: __MACOSX/raw_data/._25764.json  
  inflating: raw_data/27759.json     
  inflating: __MACOSX/raw_data/._27759.json  
  inflating: raw_data/24876.json     
  inflating: __MACOSX/raw_data/._24876.json  
  inflating: raw_data/30970.json     
  inflating: __MACOSX/raw_data/._30970.json  
  inflating: raw_data/20589.json     
  inflating: __MACOSX/raw_data/._20589.json  
  inflating: raw_data/25271.json     
  inflating: __MACOSX/raw_data/._25271.json  
  inflating: raw_data/20073.json     
  inflating: __MACOSX/raw_data/._20073.json  
  inflating: raw_data/24899.json     
  inflating: __MACOSX/raw_data/._24899.json  
  inflating: raw_data/17300.json     
  inflating: __MACOSX/raw_data/._17300.json  
  inflating: raw_data/28443.json     
  inflating: __MACOSX/raw_data/._28443.json  
  inflating: raw_data/256

In [0]:
# import packages
import os
import json

%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np


In [4]:
# record file names
repos = ['raw_data']

repo_files = []
for repo in repos:
    files = os.listdir(repo)
    files = [file for file in files if 'json' in file]
    
    repo_files.append(files)

    print('There are '+ str(len(files)) +' files in '+repo )

There are 14908 files in raw_data


In [0]:
# load tags and text information of the files
# encoding: utf-8

repo = repos[0]
files = repo_files[0]

#df = pd.DataFrame()
#
#for file in files:
#    data = pd.read_json(os.path.join(repo, file))
#    data['file_name'] = file
#    df = df.append(data, ignore_index = True)

tag_type = 17
define_columns = ['id', 'text', 'tag_0', 'tag_1', 'tag_2', 'tag_3', 'tag_4', 'tag_5', 'tag_6',
                 'tag_7', 'tag_8', 'tag_9', 'tag_10', 'tag_11', 'tag_12', 'tag_13', 'tag_14', 'tag_15', 'tag_16']

data_list = []
for file in files:
    
    with open(os.path.join(repo, file), 'r') as f:
        data = json.load(f)
        
    tags_list = [0]*tag_type
    
    for tag in data['tags']:
        tags_list[tag] = 1
    
    # TBD: using file name or original id for modeling id
    #data_list.append([data['id'], data['text']] + tags_list)
    data_list.append([file, data['text']] + tags_list)

df_data = pd.DataFrame(data_list, columns=define_columns)  

In [0]:
# output dataframe to csv for next-time convenience
df_data.to_csv('full_data.csv')


In [6]:
df_data.head()

,id,text,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,tag_9,tag_10,tag_11,tag_12,tag_13,tag_14,tag_15,tag_16
0,17791.json,退休警消- 新唐人亞太電視台\t蔡英文忠烈祠祭祀退休警消動員百人抗議. 2017-03-29...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,23296.json,十種我們常常吃的致癌食物您知多少?再忙都要耐心看完它喔！\n\nhttps://youtu....,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,17531.json,星巴克最大丑聞曝光，全球媒體刷屏！我們喝進嘴裡的咖啡，竟然都是這種東西.....\nhttp...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,20891.json,http://big5.zhengjian.org/node/230492,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,29271.json,美國爆發大規模蘿蔓生菜感染，造成五人腎衰竭，台灣有自污染地區進口，流向未知\n2019.11...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
df_data['tag_1'].sum()

142

## BERT multi-class modeling

In [7]:
!pip install keras-bert
!pip install keras-rectified-adam

  Created wheel for keras-bert: filename=keras_bert-0.81.0-cp36-none-any.whl size=37913 sha256=28db3dc5f3a42e4562d6fcc9e58b30071fed2f13522ab508248e7ca34c45ccbb
  Stored in directory: /root/.cache/pip/wheels/bd/27/da/ffc2d573aa48b87440ec4f98bc7c992e3a2d899edb2d22ef9e
  Created wheel for keras-transformer: filename=keras_transformer-0.32.0-cp36-none-any.whl size=13266 sha256=0b4fa9053442a21d2c28d7e16adeea1d41537ccdf75c901ccf848c65b58e254c
  Stored in directory: /root/.cache/pip/wheels/62/f0/ce/82fa5d024d5ef8e263f26a50dcee23820efe245680ce9c922a
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp36-none-any.whl size=7554 sha256=4f8f2626da40de1ef7e932b2ecf45869d7017136de5c13bb53a9fcce9497fe47
  Stored in directory: /root/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Created wheel for keras-multi-head: filename=keras_multi_head-0.22.0-cp36-none-any.whl size=15371 sha256=ec474f533317906cd40f4afc33dfdd9b3b149193a8ccb6c9cd5ad1d80a281ddb
  Stor

In [8]:
!wget https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip

--2020-04-17 09:47:47--  https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 2607:f8b0:400e:c09::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 381892918 (364M) [application/zip]
Saving to: ‘chinese_L-12_H-768_A-12.zip’

chinese_L-12_H-768_ 100%[===================>] 364.20M  65.4MB/s    in 6.1s    

2020-04-17 09:47:53 (59.4 MB/s) - ‘chinese_L-12_H-768_A-12.zip’ saved [381892918/381892918]



In [9]:
!unzip -o chinese_L-12_H-768_A-12.zip

Archive:  chinese_L-12_H-768_A-12.zip
   creating: chinese_L-12_H-768_A-12/
  inflating: chinese_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: chinese_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: chinese_L-12_H-768_A-12/vocab.txt  
  inflating: chinese_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: chinese_L-12_H-768_A-12/bert_config.json  


In [10]:
# import packages
%tensorflow_version 1.x
import codecs
import tensorflow as tf
from tqdm import tqdm
from chardet import detect
import keras
from keras_radam import RAdam
from keras import backend as K
from keras_bert import load_trained_model_from_checkpoint

TensorFlow 1.x selected.


Using TensorFlow backend.


In [11]:
tf.__version__

'1.15.2'

In [0]:
SEQ_LEN = 128
BATCH_SIZE = 16
EPOCHS = 3
LR = 1e-4

In [0]:
pretrained_path = '/content/chinese_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path =  os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path =  os.path.join(pretrained_path, 'vocab.txt')

In [14]:
model = load_trained_model_from_checkpoint(
      config_path,
      checkpoint_path,
      training=True,
      trainable=True,
      seq_len=SEQ_LEN,
  )

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [15]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 128)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 128)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 128, 768), ( 16226304    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 128, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [0]:
import codecs
from keras_bert import Tokenizer
token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)

In [0]:
tokenizer = Tokenizer(token_dict)

In [0]:
def load_data(df_dataset):
    global tokenizer
    indices, labels = [], []
    for row in df_dataset.iterrows():
      text = row[1]['text']
      ids, segments = tokenizer.encode(text, max_len=SEQ_LEN)
      #label = row[1].iloc[2:]
      label = list(row[1].iloc[2:])
      label = label.index(max(label))
      indices.append(ids)
      labels.append(label)
    items = list(zip(indices, labels))
    
    np.random.shuffle(items)
    test_items = items[int(0.8*len(items)):]
    train_items = items[:int(0.8*len(items))]

    indices_test, labels_test = zip(*test_items)
    indices_train, labels_train = zip(*train_items)

    indices_test = np.array(indices_test)
    indices_train = np.array(indices_train)
    
    mod_train = indices_train.shape[0] % BATCH_SIZE
    mod_test = indices_test.shape[0] % BATCH_SIZE

    if mod_train > 0:
        indices_train, labels_train = indices_train[:-mod_train], labels_train[:-mod_train]
    if mod_test > 0:
      indices_test, labels_test = indices_test[:-mod_test], labels_test[:-mod_test]

    return [indices_train, np.zeros_like(indices_train)], np.array(labels_train), [indices_test, np.zeros_like(indices_test)], np.array(labels_test)
  
train_x, train_y, test_x, test_y = load_data(df_data)

In [19]:
test_y[3]

14

In [0]:
inputs = model.inputs[:2]
dense = model.get_layer('NSP-Dense').output
outputs = keras.layers.Dense(units=tag_type, activation='softmax')(dense)

model = keras.models.Model(inputs, outputs)
model.compile(
  RAdam(learning_rate =LR),
  loss='sparse_categorical_crossentropy',
  metrics=['sparse_categorical_accuracy'],
)

In [21]:
sess = K.get_session()
uninitialized_variables = set([i.decode('ascii') for i in sess.run(tf.report_uninitialized_variables())])
init_op = tf.variables_initializer(
    [v for v in tf.global_variables() if v.name.split(':')[0] in uninitialized_variables]
)
sess.run(init_op)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [22]:
model.fit(
    train_x,
    train_y,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
)

Epoch 1/3
11920/11920 [==============================] - 221s 19ms/step - loss: 1.0252 - sparse_categorical_accuracy: 0.6920
Epoch 2/3
11920/11920 [==============================] - 209s 18ms/step - loss: 0.6826 - sparse_categorical_accuracy: 0.7900
Epoch 3/3
11920/11920 [==============================] - 209s 18ms/step - loss: 0.5716 - sparse_categorical_accuracy: 0.8246


## Prediction

In [27]:
test_predicts = model.predict(test_x, verbose=True).argmax(axis=-1)
#predicts = model.predict(train_x, verbose=True).argmax(axis=-1)

2976/2976 [==============================] - 14s 5ms/step


In [28]:
train_predicts = model.predict(train_x, verbose=True).argmax(axis=-1)

11920/11920 [==============================] - 55s 5ms/step


## Evaluation metrics

In [0]:
# import sklearn metrics packages
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.metrics import classification_report

In [29]:
print("Training Accuracy of BERT is:", accuracy_score(train_y, train_predicts))
print("Testing Accuracy of BERT is:", accuracy_score(test_y, test_predicts))

Training Accuracy of BERT is: 0.885234899328859
Testing Accuracy of BERT is: 0.7614247311827957


In [30]:
print(classification_report(train_y, train_predicts))
print(classification_report(test_y, test_predicts))

              precision    recall  f1-score   support

           0       0.74      0.90      0.81       428
           1       0.54      0.90      0.67       110
           2       0.87      0.68      0.76        50
           3       0.97      0.97      0.97      2738
           4       0.00      0.00      0.00        12
           5       1.00      0.43      0.60        42
           6       0.62      0.78      0.69        83
           7       0.67      0.63      0.65        71
           8       0.86      0.88      0.87       768
           9       0.77      0.87      0.82       280
          10       0.84      0.96      0.89       668
          11       0.83      0.84      0.83      1444
          12       0.95      0.48      0.64       518
          13       0.91      0.60      0.72       339
          14       0.93      0.93      0.93      3311
          15       0.86      0.91      0.89       969
          16       0.65      0.91      0.76        89

    accuracy              

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
sum(train_y==3)

2768

In [0]:
model.save_weights('bert_weights_20200416.ckpt')

In [0]:
model.save_weights('bert_20200416.h5')

In [0]:
from keras.models import load_model
model2 = load_model('bert_20200416.h5')